without bigram

In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import fasttext
import nltk
from nltk.stem import WordNetLemmatizer

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load the IMDB dataset
df = pd.read_csv("IMDB Dataset.csv")

# Rename the columns if needed (this is just a check to ensure compatibility)
df.columns = ['review', 'sentiment']

# Check for missing values and drop them if necessary
df = df.dropna(subset=['review', 'sentiment'])

# Preprocess the text
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Remove non-alphabetic characters
    tokens = word_tokenize(text)  # Tokenize the text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Remove stopwords and lemmatize
    return ' '.join(tokens)

df['cleaned_review'] = df['review'].apply(preprocess_text)

# Remove empty rows after preprocessing
df = df[df['cleaned_review'].str.strip() != '']

# Format the labels for FastText
def format_labels(sentiment):
    return f"__label__{sentiment}"

df['formatted_labels'] = df['sentiment'].apply(format_labels)

# Split the dataset into training and testing (80% train, 20% test)
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

# Save training and test data for FastText
with open("train_imdb.txt", "w") as f:
    for label, review in zip(train_df['formatted_labels'], train_df['cleaned_review']):
        f.write(f"{label} {review}\n")

with open("test_imdb.txt", "w") as f:
    for label, review in zip(test_df['formatted_labels'], test_df['cleaned_review']):
        f.write(f"{label} {review}\n")

# Check if the training file has valid content
with open("train_imdb.txt", "r") as f:
    for i in range(5):
        print(f.readline())

# Train the FastText model with the IMDB dataset
model = fasttext.train_supervised(
    input="train_imdb.txt", 
    dim=10, 
)

# Save the model
model.save_model("imdb_model.bin")

# Test the FastText model
result = model.test("test_imdb.txt")
print(f"Number of examples: {result[0]}")
print(f"Precision (Accuracy): {result[1]}")
print(f"Recall: {result[2]}")

# Calculate and print accuracy
accuracy = result[1]  # Precision is the accuracy
print(f"Test Accuracy: {accuracy:.4f}")


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/puneethreddy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/puneethreddy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/puneethreddy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


__label__positive really liked summerslam due look arena curtain look overall interesting reason anyways could one best summerslams ever wwf didnt lex luger main event yokozuna time ok huge fat man v strong man im glad time changed terrible main event like every match luger terrible match card razor ramon v ted dibiase steiner brother v heavenly body shawn michael v curt hening event shawn named big monster body guard diesel irs v kid bret hart first take doink take jerry lawler stuff hart lawler always interesting ludvig borga destroyed marty jannetty undertaker took giant gonzalez another terrible match smoking gunns tatanka took bam bam bigelow headshrinkers yokozuna defended world title lex luger match boring terrible ending however deserves

__label__positive many television show appeal quite many different kind fan like farscape doesi know youngster year oldfans male female many different country think adore tv miniseries element found almost every show tv character driven drama 

Read 4M words
Number of words:  145012
Number of labels: 2
Progress:  82.3% words/sec/thread: 9226142 lr:  0.017706 avg.loss:  0.289183 ETA:   0h 0m 0s

Number of examples: 10000
Precision (Accuracy): 0.8847
Recall: 0.8847
Test Accuracy: 0.8847


Progress: 100.0% words/sec/thread: 8406812 lr:  0.000000 avg.loss:  0.275581 ETA:   0h 0m 0s


with bigram

In [3]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import fasttext
import nltk
from nltk.stem import WordNetLemmatizer

# Download required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Load the IMDB dataset
df = pd.read_csv("IMDB Dataset.csv")

# Rename the columns if needed (this is just a check to ensure compatibility)
df.columns = ['review', 'sentiment']

# Check for missing values and drop them if necessary
df = df.dropna(subset=['review', 'sentiment'])

# Preprocess the text
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)  # Remove non-alphabetic characters
    tokens = word_tokenize(text)  # Tokenize the text
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]  # Remove stopwords and lemmatize
    return ' '.join(tokens)

df['cleaned_review'] = df['review'].apply(preprocess_text)

# Remove empty rows after preprocessing
df = df[df['cleaned_review'].str.strip() != '']

# Format the labels for FastText
def format_labels(sentiment):
    return f"__label__{sentiment}"

df['formatted_labels'] = df['sentiment'].apply(format_labels)

# Split the dataset into training and testing (80% train, 20% test)
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

# Save training and test data for FastText
with open("train_imdb.txt", "w") as f:
    for label, review in zip(train_df['formatted_labels'], train_df['cleaned_review']):
        f.write(f"{label} {review}\n")

with open("test_imdb.txt", "w") as f:
    for label, review in zip(test_df['formatted_labels'], test_df['cleaned_review']):
        f.write(f"{label} {review}\n")

# Check if the training file has valid content
with open("train_imdb.txt", "r") as f:
    for i in range(5):
        print(f.readline())

# Train the FastText model with bigrams (wordNgrams=2)
model = fasttext.train_supervised(
    input="train_imdb.txt", 
    dim=10, 
    wordNgrams=2  # Enable bigrams
)

# Save the model
model.save_model("imdb_model_bigrams.bin")

# Test the FastText model
result = model.test("test_imdb.txt")
print(f"Number of examples: {result[0]}")
print(f"Precision (Accuracy): {result[1]}")
print(f"Recall: {result[2]}")

# Calculate and print accuracy
accuracy = result[1]  # Precision is the accuracy
print(f"Test Accuracy: {accuracy:.4f}")


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/puneethreddy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/puneethreddy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/puneethreddy/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


__label__positive really liked summerslam due look arena curtain look overall interesting reason anyways could one best summerslams ever wwf didnt lex luger main event yokozuna time ok huge fat man v strong man im glad time changed terrible main event like every match luger terrible match card razor ramon v ted dibiase steiner brother v heavenly body shawn michael v curt hening event shawn named big monster body guard diesel irs v kid bret hart first take doink take jerry lawler stuff hart lawler always interesting ludvig borga destroyed marty jannetty undertaker took giant gonzalez another terrible match smoking gunns tatanka took bam bam bigelow headshrinkers yokozuna defended world title lex luger match boring terrible ending however deserves

__label__positive many television show appeal quite many different kind fan like farscape doesi know youngster year oldfans male female many different country think adore tv miniseries element found almost every show tv character driven drama 

Read 4M words
Number of words:  145012
Number of labels: 2
Progress: 100.0% words/sec/thread: 5470999 lr:  0.000000 avg.loss:  0.298115 ETA:   0h 0m 0s


Number of examples: 10000
Precision (Accuracy): 0.8863
Recall: 0.8863
Test Accuracy: 0.8863
